In [1]:
# Importing metadata zip file and converting it to dataframe

import json
import gzip
import pandas as pd

def getDF(path):
  # Unzip the file, load in each line as an object
  g = gzip.open(path, 'rb')
  g = [json.loads(l) for l in g]

  # Map to a dictionary, then load in as a dataframe
  dict_df = {i: d for (i, d) in enumerate(g)}
  return pd.DataFrame.from_dict(dict_df, orient='index')

df = getDF('meta_ALL_Beauty.json.gz')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_38108\2769020576.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

# Selecting only the colums that are required for analysis

colums_description_asin = ["description","asin"]
df = df[colums_description_asin]

df.describe()

,description,asin
count,32892,32892
unique,13751,32488
top,[],B00027CDOW
freq,17773,2


In [3]:
# Descriptions in the json file are a list of strings, converting to sting for easy cleaning
df["newdescription"] = df.description.map(lambda x: ".".join(x).replace("\n",""))


In [4]:
# Remove empty descriptions
df = df.drop(df[df["newdescription"] == ""].index)

print(len(df)) 



15108


In [5]:
# Remove duplicate asin so can have only unique products
df[df.duplicated("asin")]
df = df.drop_duplicates("asin", keep="last")
df.describe()

,description,asin,newdescription
count,14821,14821,14821
unique,13749,14821,13743
top,"[For over 60 years, Betty Dain Creations, Inc....",6546546450,"For over 60 years, Betty Dain Creations, Inc. ..."
freq,59,1,59


In [6]:
# Remove duplicate description
df[df.duplicated("newdescription")]
df = df.drop_duplicates("newdescription", keep="last")
df.describe() 

,description,asin,newdescription
count,13743,13743,13743
unique,13743,13743,13743
top,[Loud 'N Clear Personal Sound Amplifier allows...,6546546450,Loud 'N Clear Personal Sound Amplifier allows ...
freq,1,1,1


In [7]:
# Removing redundant old description column
colums_description_asin = ["newdescription","asin"]
df = df[colums_description_asin]

df.describe()

,newdescription,asin
count,13743,13743
unique,13743,13743
top,Loud 'N Clear Personal Sound Amplifier allows ...,6546546450
freq,1,1


In [10]:
# Remove descriptions with more than 511 pre-cleaned words

# Split at any white space 
df["num_words_description"] = df["newdescription"].apply(lambda x: len(x.split()))

# Check if under or equal to 511 words fulfils withs condition and set it
df = df[df["num_words_description"] <= 511]

df["newdescription"].describe()

count                                                 13688
unique                                                13688
top       Loud 'N Clear Personal Sound Amplifier allows ...
freq                                                      1
Name: newdescription, dtype: object

In [24]:
# Regrex for character removal
import re

# Spacy for spell check
import spacy
import contextualSpellCheck
nlp = spacy.load("en_core_web_sm")
contextualSpellCheck.add_to_pipe(nlp)

# NLTK for tokenisation, remocing stop words and lemmatization
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('stopwords')

# Preprocessing of descriptions

def preprocessing(raw_string):
    # Remove html tags and anything inside them 
    no_html = re.sub(r'<[^>]*>','', raw_string)
    #print("after removing html", no_html)


    # Make everything lowercase
    lowercase_column = no_html.lower()
    #print("lowercase", lowercase_column)

    # Remove apostrophe to enable spell check to correct words with apostrophe
    without_apostrophe = re.sub(r'[\']', '', lowercase_column)

    # ! Need to double check again where best to use this spell check
    # 
    # .pepe for batches of text
    #doc = list(nlp.pipe(without_apostrophe))

    #spell_checked = doc._.outcome_spellCheck

    # Remove all non alphabetic instances that aren't a space and replace them with a space using Regrex
    alphabetic_column = re.sub(r'[^a-z\s]', ' ', without_apostrophe)
    #print("removed numerical and punctuation", alphabetic_column)

    tokens = word_tokenize(alphabetic_column)

    return tokens

df["clean_description"] = df["newdescription"].head(5).apply(preprocessing)

df["clean_description"]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        [loud, n, clear, personal, sound, amplifier, a...
1        [no, lift, luminate, triple, action, serum, ml...
2        [no, stay, perfect, foundation, now, stays, pe...
4        [lacto, calamine, skin, balance, daily, nouris...
5        [mary, kay, satin, hands, peach, hand, cream, ...
                               ...                        
32880                                                  NaN
32884                                                  NaN
32885                                                  NaN
32886                                                  NaN
32890                                                  NaN
Name: clean_description, Length: 13688, dtype: object